In [2]:
import networkx as nx
import time
from networkx.algorithms.community.quality import modularity
from networkx.algorithms.community import louvain_communities
from networkx.algorithms.community.centrality import girvan_newman
from collections import Counter
import pandas as pd
from sklearn.metrics import normalized_mutual_info_score

# Load the graph from a text file
file_path = '/Users/sabinanurseitova/Desktop/SNA/friends_episodes.txt'
G = nx.Graph()
# opening the file and adding edges to the graph

try:
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):#skipping empty lines and comments
                continue  
            node1, node2 = line.split()#splitting the line into 2 nodes
            G.add_edge(node1, node2)
except Exception as e:
    print(f"Error loading file: {e}")
    exit()
    
G.remove_edges_from(nx.selfloop_edges(G))# Removing self-loops
largest_cc = max(nx.connected_components(G), key=len)# Keeping only the largest connected component of the graph, so we can focus on the main part of the network and ignore smaller, less meaningful parts.
G = G.subgraph(largest_cc).copy()

# Function to gn for community detection
def girvan_newman_communities(graph):
    try:
        G_copy = graph.copy()#We work on a copy here to avoid making modifications to an original graph
        best_modularity = -1 #starting with a very low modularity value to ensure any calculated modularity will update the best value.
        best_partition = None #storing the best partition found
        start_time = time.time()# to record the starting time for computation, so we can use it for analysis later
    
        comp = girvan_newman(G_copy) #generation of communities iteratively 
        for communities in comp:
            partition= [list(c) for c in communities]
            current_modularity = modularity(graph, partition)
            if current_modularity > best_modularity:
                best_modularity = current_modularity
                best_partition = partition
        end_time = time.time()#recording of the end time for further analysis
        computation_time = end_time - start_time# telling how to compute the computation time 
        return best_partition, best_modularity, computation_time
    except Exception as e:
        print(f"Error in Girvan - Newman: {e}")
        return None, -1, -1
# Function to apply Louvain method for community detection
def modularity_optimization_communities(graph):
    try:
        start_time = time.time()#recording starting time for further analysis
        communities = louvain_communities(graph, weight=None, resolution=1)#Finding communities in the graph, ignoring edge weights and with the standard resolution setting of 1.
        end_time = time.time()# end time recording 
        modularity_score = modularity(graph, communities)#calculating modularity for the detected communities
        computation_time = end_time - start_time #computation time calculation 
        return communities, modularity_score, computation_time
    except Exception as e:
        print(f"Error in Louvain: {e}")
        return None, -1, -1
# Function to apply Label Propagation method for community detection
def label_propagation_communities(graph):
    try: 
        start_time = time.time()#start time recording 
        communities = list(nx.algorithms.community.label_propagation_communities(graph))#detecting communitites
        end_time = time.time()#end time recording 
        modularity_score = modularity(graph, communities)#calculation of modularity for this method
        computation_time = end_time - start_time#computation time calculation 
        return communities, modularity_score, computation_time
    except Exception as e:
        print(f"Error in Label Propagation: {e}")
        return None, -1, -1
# Function to compute Normalized Mutual Information (NMI) between two partitions
def compute_nmi(partition1, partition2):
    # Flattening the partitions so that each node is assigned to its community index
    try:
        partition1_flat = [i for idx, community in enumerate(partition1) for i in [idx] * len(community)]
        partition2_flat = [i for idx, community in enumerate(partition2) for i in [idx] * len(community)]
        return normalized_mutual_info_score(partition1_flat, partition2_flat)
    except Exception as e:
        print(f"Error in NMI calculation: {e}")
        return -1

#Here, we are telling the code to apply the previous community detection methods
gn_partition, gn_modularity, gn_time = girvan_newman_communities(G)
louvain_partition, louvain_modularity, louvain_time = modularity_optimization_communities(G)
label_partition, label_modularity, label_time = label_propagation_communities(G)
#outputs verification 
print(f"Girvan-Newman: {gn_partition}, Modularity: {gn_modularity}, Time: {gn_time}")
print(f"Louvain: {louvain_partition}, Modularity: {louvain_modularity}, Time: {louvain_time}")
print(f"Label Propagation: {label_partition}, Modularity: {label_modularity}, Time: {label_time}")

#Here, we are telling the code to compute NMI between the partitions detected by those 3 methods
nmi_gn_louvain = compute_nmi(gn_partition, louvain_partition)
nmi_gn_label = compute_nmi(gn_partition, label_partition)
nmi_louvain_label = compute_nmi(louvain_partition, label_partition)

#Setting a format for the results to be displayed 
results = {
    "Method": ["Girvan - Newman", "Modularity Optimization (Louvain)", "Label Propagation"],
    "Num Clusters": [len(gn_partition), len(louvain_partition), len(label_partition)],
    "Cluster Size Distribution": [
        Counter(len(cluster) for cluster in gn_partition),
        Counter(len(cluster) for cluster in louvain_partition),
        Counter(len(cluster) for cluster in label_partition)
    ],
    "Modularity": [gn_modularity, louvain_modularity, label_modularity],
    "Computation Time (s)": [gn_time, louvain_time, label_time],
    "NMI (GN vs Louvain)": [nmi_gn_louvain, "N/A", "N/A"],
    "NMI (GN vs Label)": [nmi_gn_label, "N/A", "N/A"],
    "NMI (Louvain vs Label)": ["N/A", nmi_louvain_label, "N/A"]
}

results_df = pd.DataFrame(results)# Converting the results into a DataFrame for easy reading
print(results_df)


# Verify the graph after loading
print(f"Graph loaded with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

# Verify the largest connected component
if G.number_of_nodes() == 0:
    print("Error: Graph is empty after loading!")
    exit()

# Verify community detection results
print("Running Girvan-Newman...")
gn_partition, gn_modularity, gn_time = girvan_newman_communities(G)
print(f"Girvan-Newman results: {len(gn_partition)} communities, Modularity: {gn_modularity}, Time: {gn_time}")

print("Running Louvain...")
louvain_partition, louvain_modularity, louvain_time = modularity_optimization_communities(G)
print(f"Louvain results: {len(louvain_partition)} communities, Modularity: {louvain_modularity}, Time: {louvain_time}")

print("Running Label Propagation...")
label_partition, label_modularity, label_time = label_propagation_communities(G)
print(f"Label Propagation results: {len(label_partition)} communities, Modularity: {label_modularity}, Time: {label_time}")

# Verify NMI computation
if gn_partition and louvain_partition:
    nmi_gn_louvain = compute_nmi(gn_partition, louvain_partition)
    print(f"NMI between Girvan-Newman and Louvain: {nmi_gn_louvain}")
else:
    print("Error: Invalid partitions for NMI computation (GN vs Louvain)")

if gn_partition and label_partition:
    nmi_gn_label = compute_nmi(gn_partition, label_partition)
    print(f"NMI between Girvan-Newman and Label Propagation: {nmi_gn_label}")
else:
    print("Error: Invalid partitions for NMI computation (GN vs Label)")

if louvain_partition and label_partition:
    nmi_louvain_label = compute_nmi(louvain_partition, label_partition)
    print(f"NMI between Louvain and Label Propagation: {nmi_louvain_label}")
else:
    print("Error: Invalid partitions for NMI computation (Louvain vs Label)")

# Ensure final results are valid
if gn_partition and louvain_partition and label_partition:
    results_df = pd.DataFrame(results)
    print("Final Results:")
    print(results_df)
else:
    print("Error: Could not generate final results due to invalid community detection outputs.")


Girvan-Newman: [['Wendy', 'Monica', 'neighborWoman', 'doctor_10_17', 'waiter_2_5', 'neighbor_7_1', "Chandler'sAssistant", 'Marjorie', 'intern', 'TomGordon', 'Jeanette', 'Dan', 'bandLeader', 'mailman', 'airlineEmpl', 'Tomas', 'Gandalf', 'reunionMember', 'Ellen', 'MrsBecker', 'Santos', 'Ken', 'womanHoneymoon', 'twinCarl', 'waitress_7_7', 'nurse', 'deskClerk', 'Drew', 'receptionist_9_19', 'moverGuy2', 'Hildy', 'bride', 'MrCostilick', 'clerk', "Frannie'sFriend", 'neighbor', 'Marcel', 'Owen', 'MrTyler', 'MsMcKenna', 'Karen', 'SandraGreen', 'Dana', 'Paul', 'Brenda', 'man@wedding', 'restManager', 'Colleen', 'realtor', 'tktcterAttendant', 'Danielle', "Frannie'sMate", 'Tilly', 'olderGuest', 'NonnaTribbiani', 'nurse_9_21', 'adoptionAgencyWorker', 'MrTribbiani', 'saleswoman', 'partyGoers', 'Syl', 'Eddie', 'Erika', 'Paolo', 'moverGuy1', 'Bill_10', 'Chandler', 'Roger', 'Alan', 'MrBuddyDoyle', 'photographer', 'LeonRastatter', 'hooker', 'Parker', 'Frankie', 'Kori', 'loungeGuy', 'nurse3', 'Gail', 'MsT

GN Method: Clusters: 39, many small clusters with one large component (415 nodes).
Modularity: 0.3923, which is moderate, indicating a somewhat weak community separation.
Computation Time: Took 1040.34 seconds, which is significantly longer than the other methods.
NMI: Moderate agreement with Louvain (0.6570), weaker with Label Propagation (0.3020).

Louvain Method: Clusters: 10, fewer and more cohesive communities.
Modularity: 0.4069, which is high compared to the other methods, suggesting well-defined, cohesive communities.
Computation Time: Fastest at 0.0633 seconds.
NMI: Moderate agreement with Girvan-Newman (0.1228), but low with Label Propagation (0.1228).

Label Propagation Method: Clusters: 21, moderate fragmentation.
Modularity: 0.0495, lower than the Louvain method, suggesting less cohesive communities.
Computation Time: Fastest at 0.0336 seconds.
NMI: Low agreement with both Girvan-Newman and Louvain.

Louvain is the most effective method for community detection here, providing well-formed communities and doing so efficiently.


In [6]:
import networkx as nx

# Step 1: Creating a graph (you can replace this with your own graph)
G = nx.Graph()
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5)])  # Just adding some random connections

# Step 2: Using the Louvain method to find groups (communities) in the graph
def modularity_optimization_communities(graph):
    # This is finding all the groups of nodes in the graph
    return list(nx.community.louvain_communities(graph))

louvain_partition = modularity_optimization_communities(G)  # Getting the groups in the graph

# Step 3: Creating a dictionary to keep track of which node belongs to which group
louvain_partition_dict = {}  
for idx, community in enumerate(louvain_partition):  # Looping through each group
    for node in community:  # Looping through each node in the group
        louvain_partition_dict[node] = idx  # Storing which group the node is in

# Step 4: Adding the group info to each node in the graph
for node in G.nodes():  # Looping through all nodes in the graph
    G.nodes[node]['community'] = louvain_partition_dict[node]  # Tagging each node with its group

# Step 5: Saving the graph with the group info so we can open it in Gephi
output_file = "/Users/sabinanurseitova/Desktop/SNA/louvain_partition.gexf"
nx.write_gexf(G, output_file)  # Saving the graph to a file
print(f"Louvain partition saved to {output_file}")  # Letting us know where it got saved


Louvain partition saved to /Users/sabinanurseitova/Desktop/SNA/louvain_partition.gexf


The Girvan-Newman method found a lot of clusters (39), but the communities it found weren’t very strong or separate from each other. The modularity score was pretty low (0.3923), meaning the communities didn’t have much structure. Plus, it took a really long time (1040.34 seconds), so it’s not the most efficient method.

The Louvain method found fewer clusters (10), but the communities were much more defined and stuck together better, with a higher modularity score (0.4069). It also worked super fast (0.0633 seconds), which makes it much more practical for larger graphs.

Label Propagation found 21 clusters, but these communities were pretty weak, with a very low modularity score (0.0495). While it was the fastest method (0.0336 seconds), it didn’t do a good job at detecting strong communities.